In [4]:
import findspark
findspark.init()

In [33]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import *
import pickle
import pyhdfs
import pyspark
import pandas as pd
import numpy as np
import os

In [6]:
spark =SparkSession.builder.getOrCreate()

In [20]:
MRI_df = spark.read.csv("hdfs://quickstart.cloudera/user/cloudera/model_deploy/data/Pdata.txt",header=True,inferSchema=True)

In [21]:
MRI_df.printSchema()

root
 |-- PNO: integer (nullable = true)
 |-- ITEM_n: integer (nullable = true)
 |-- ORDERDR_n: integer (nullable = true)
 |-- DEPT_n: integer (nullable = true)
 |-- SEX_n: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- POS_n: integer (nullable = true)
 |-- IO_n: integer (nullable = true)
 |-- SECOND: integer (nullable = true)



In [22]:
MRI_df.show()

+--------+------+---------+------+-----+---+-----+----+------+
|     PNO|ITEM_n|ORDERDR_n|DEPT_n|SEX_n|AGE|POS_n|IO_n|SECOND|
+--------+------+---------+------+-----+---+-----+----+------+
|37753240|     0|      197|    22|    0| 64|   17|   1|  2296|
|21426434|     0|       66|     4|    0| 76|    2|   1|  2517|
|50248995|     0|       72|     5|    0| 22|    0|   1|   532|
|46645466|     0|      271|    22|    0| 20|   15|   1|  1807|
|24694872|     0|       72|     5|    1| 67|    6|   1|  1445|
|27828025|     0|       95|     4|    0| 67|   12|   1|  2318|
|23183318|     0|       73|     5|    0| 84|    0|   1|  4856|
|51684790|     0|       47|     5|    0| 43|    0|   1|    23|
| 9718999|     0|      136|    17|    0| 76|   12|   1|   808|
|25109861|     0|       72|     5|    1| 53|    0|   1|    23|
|41518866|     0|       53|     4|    1| 65|    0|   1|  1176|
|33843672|     0|       73|     5|    1| 38|    4|   1|  1608|
|51680463|     0|       72|     5|    0| 50|    1|   1|

### textfile import to DF

In [24]:
sc = spark.sparkContext

In [27]:
data = sc.textFile('hdfs://quickstart.cloudera/user/cloudera/model_deploy/data/Pdata_rdd.txt')
parts = data.map(lambda l :l.split(","))

In [30]:
parts.take(5)

[['37753240', '0', '197', '22', '0', '64', '17', '1', '2296'],
 ['21426434', '0', '66', '4', '0', '76', '2', '1', '2517'],
 ['50248995', '0', '72', '5', '0', '22', '0', '1', '532'],
 ['46645466', '0', '271', '22', '0', '20', '15', '1', '1807'],
 ['24694872', '0', '72', '5', '1', '67', '6', '1', '1445']]

In [44]:
people = parts.map(lambda p: Row(PNO = p[0],Item = int(p[1]),ORDERDR = int(p[2]),DEPT = int(p[3]),SEX = int(p[4]),AGE = int(p[5]),POS = int(p[6]),IO = int(p[7]),SECOND = int(p[8])))

In [46]:
schemaPeople = people.toDF()

In [48]:
schemaPeople.printSchema()

root
 |-- AGE: long (nullable = true)
 |-- DEPT: long (nullable = true)
 |-- IO: long (nullable = true)
 |-- Item: long (nullable = true)
 |-- ORDERDR: long (nullable = true)
 |-- PNO: string (nullable = true)
 |-- POS: long (nullable = true)
 |-- SECOND: long (nullable = true)
 |-- SEX: long (nullable = true)



In [49]:
schemaPeople.createOrReplaceTempView("people")

In [55]:
test = spark.sql("select PNO,SECOND From people where AGE > 20 and IO = 2")

In [56]:
test.show()

+--------+------+
|     PNO|SECOND|
+--------+------+
|30429971|  1860|
|51647441|  2720|
|51477931|  3123|
|36984811|  1138|
|34264946|  2947|
|  971625|  1883|
|29120361|  2404|
|10036256|  2595|
|32139490|   173|
|12691530|  1493|
|50238389|  3172|
|14301733|  2556|
|37612764|  2872|
|51958062|  4750|
|39930297|   216|
|41382121|  2184|
|27160449|  1983|
|51965885|  2825|
|45419324|  2148|
|26953974|   137|
+--------+------+
only showing top 20 rows



In [61]:
schemaPeople.write.csv('hdfs://quickstart.cloudera/user/cloudera/model_deploy/data/pdata_sparksql',mode='append')

In [62]:
schemaPeople.write.parquet('hdfs://quickstart.cloudera/user/cloudera/model_deploy/data/pdata_sparksql',mode='append')